# <b><font color='#009e74'>Absolute folding stabiity prediction via generative models</font></b>
Colaboratory implementation of : **Cagiada M., Ovchinnikov S. & Lindorff-Larsen K.** - [Predicting absolute protein folding stability using generative models
](https://doi.org/10.1038/s41467-023-39909-0). Source code is available on the [Github](https://github.com/KULL-Centre/_2024_cagiada_stability) page of the project.

Prediction of the absolute stability for a target folding given a protein sequence.

The Colab uses ESM-IF and the following measure:

$\Delta G_{f-u} = \sum_{i}^{N} \mathscr{L}_i^{WT}$

where $\mathscr{L}_i^{WT}$ is the amino acid likelihood extracted from ESM-IF for the wild-type amino acid at position i, to evaluate the absolute stability ($\Delta G_{f-u}$) for a specific protein folding.



Additional notes:

- Run this notebook  <b><font color='#d55c00'>preferabilly</font></b> in a Colab GPU session (go to page menu: `Runtime`->  `Change runtime type` -> select `GPU` and confirm.

- Cells labelled <b><font color='#f0e422'>PRELIMINARY OPERATIONS </font></b>  have to be run <b><font color='#d55c00'>ONCE</font></b>  at the start and skipped for new predictions.

- Multiple predictions can be run in a single session, but only <b><font color='#d55c00'>ONE</font></b> pdb at a time will be processed by the notebook.

- A <b><font color='#d55c00'>new run</font></b> can be perform input direcly the new structure in the pdb upload cell and run the prediction cell again (you don't need to run the <b><font color='#f0e422'>PRELIMINARY OPERATIONS </font></b> again.

- If you wish to download the predictions run the <b><font color='#56b4e9'>DOWNLOAD RESULTS </font></b> and <b>ALL</b>
 the predictions made in the session will be dowloaded.

- To evaluate absolute stability the model uses an input structure in PDB format, usually in the form of an [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2) predicted structure, if you have an experimentally resolved structure please consider generating an AF2 prediction using your structure as a template.

- The absolute stability can only be predicted for <b><font color='#d55c00'>one</font></b> chain at a time, but the input structure protein can be either a single chain or a multi-chain complex (the remaining chains would not be taken into account during the prediction).

- The output of a prediction is displayed below the prediction cell and saved in an output file which includes the absolute stability (as a sum of likelihoods and in kcal/mol) and the contribution to the total stability for each residue (as a likelihood).

- An **alternative sequence** can be used in the input instead of the sequence extracted from the PDB file, **HOWEVER** it must be the same length as the original sequence.
****

In [ ]:

#@title <b><font color='#f0e422'>PRELIMINARY OPERATIONS:</font> Install dependencies

#@markdown Run the cell to install all the extra necessaries packages <b>(~1min 30s)</b>, including:
#@markdown - ESM-IF (library and parameters)
#@markdown - Torch libraries: torch-scatter,-sparse,-cluster,spline-conv,-geometric
#@markdown - Python libraries: biopython, biotite

%%time
import os,time,subprocess,re,sys,shutil
from google.colab import files
import torch
import numpy as np
import pandas as pd

def format_pytorch_version(version):
  return version.split('+')[0]

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

IF_model_name = "esm_if1_gvp4_t16_142M_UR50.pt"

if not os.path.isfile(IF_model_name):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system(f"aria2c -x 16 https://sid.erda.dk/share_redirect/eIZVVNEd8B --out={IF_model_name} &")

  if not os.path.isfile("finished_install"):
    # install libs
    print("installing libs...")

    os.system(f"pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
    os.system(f"pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
    os.system(f"pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
    os.system(f"pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
    os.system(f"pip install torch-geometric")
    os.system(f"pip install biopython")
    os.system(f"pip install biotite==0.41")

    print("installing esmfold...")
    # install esmfold
    os.system(f"pip install git+https://github.com/matteo-cagiada/esm.git")
    os.system("touch finished_install")

    #wait for Params to finish downloading...
    while not os.path.isfile(IF_model_name):
      time.sleep(5)
    if os.path.isfile(f"{IF_model_name}.aria2"):
      print("downloading params...")
    while os.path.isfile(f"{IF_model_name}.aria2"):
      time.sleep(5)

## Verify that pytorch-geometric is correctly installed

import esm

from esm.inverse_folding.util import load_structure, extract_coords_from_structure,CoordBatchConverter
from esm.inverse_folding.multichain_util import extract_coords_from_complex,_concatenate_coords,load_complex_coords


print("importing the model")

model, alphabet = esm.pretrained.load_model_and_alphabet(IF_model_name)
model.eval().cuda().requires_grad_(False)

print("--> Installations succeeded")

In [ ]:
#@title <b><font color='#f0e422'>PRELIMINARY OPERATIONS:</font> Load EXTRA functions
#@markdown Run the cell to load the required functions

def run_model(coords,sequence,model,cmplx=False,chain_target='A'):

    device = next(model.parameters()).device

    batch_converter = CoordBatchConverter(alphabet)
    batch = [(coords, None, sequence)]
    coords, confidence, strs, tokens, padding_mask = batch_converter(
        batch, device=device)

    prev_output_tokens = tokens[:, :-1].to(device)
    target = tokens[:, 1:]
    target_padding_mask = (target == alphabet.padding_idx)

    logits, _ = model.forward(coords, padding_mask, confidence, prev_output_tokens)

    logits_swapped=torch.swapaxes(logits,1,2)
    token_probs = torch.softmax(logits_swapped, dim=-1)

    return token_probs

def score_variants(sequence,token_probs,alphabet):

    aa_list=[]
    wt_scores=[]
    skip_pos=0

    alphabetAA_L_D={'-':0,'_' :0,'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}
    alphabetAA_D_L={v: k for k, v in alphabetAA_L_D.items()}

    for i,n in enumerate(sequence):
      aa_list.append(n+str(i+1))
      score_pos=[]
      for j in range(1,21):
          score_pos.append(masked_absolute(alphabetAA_D_L[j],i, token_probs, alphabet))
          if n == alphabetAA_D_L[j]:
            WT_score_pos=score_pos[-1]

      wt_scores.append(WT_score_pos)

    return aa_list, wt_scores

def masked_absolute(mut, idx, token_probs, alphabet):

    mt_encoded = alphabet.get_idx(mut)

    score = token_probs[0,idx, mt_encoded]
    return score.item()

In [ ]:
#@title <b><font color='#56b4e9'> DATA UPLOADING</font>
#@markdown Fill in the fields and run the cell to set up the job name, import the structure, select the chain and upload an alternative sequence (not mandatory).
jobname='lys_ecoly'#@param {type:"string"}

#@markdown Choose between <b><font color='#d55c00'> ONE</font></b> of the possible input sources for the target pdb and <b><font color='#d55c00'>leave the other cells empty or unmarked</font></b>
#@markdown - AlphaFold2 PDB (v4) via Uniprot ID:
AF_ID ='P78285'#@param {type:"string"}
#@markdown - Upload custom PDB
AF_custom =False#@param {type:"boolean"}


#@markdown Select target chain (default A)
chain_id='A' #@param {type:'string'}

#@markdown Upload an alternative sequence for the structure (leave empty if not used)
alternative_sequence='' #@param {type:'string'}

input_path = f"/content/inputs"
if not os.path.exists(input_path):
  os.mkdir(input_path)

output_path = f"/content/outputs"
if not os.path.exists(output_path):
  os.mkdir(output_path)

if AF_custom:
  print('Upload PDB file:')
  uploaded_AF = files.upload()
  for fn in uploaded_AF.keys():
    os.rename(fn, f"/content/inputs/query_protein.pdb")
    output_name_pdb=fn
    print('... PDB file correctly loaded')
elif (AF_ID !='') and (len(AF_ID)>=6) :
    subprocess.call(['curl','-s','-f',f'https://alphafold.ebi.ac.uk/files/AF-{AF_ID}-F1-model_v4.pdb','-o','/content/inputs/query_protein.pdb'])
    output_name_pdb=f'AF-{AF_ID}-F1-model_v4.pdb'
else:
  sys.exit(f'ERROR: any PDB uploaded, please select one of the above inputs')

structure = load_structure(f"/content/inputs/query_protein.pdb", chain_id)
coords_structure, sequence_structure = extract_coords_from_structure(structure)

if alternative_sequence != '':
  if ' ' in alternative_sequence:
    sys.exit ('!!!! Run interrupted: please check input sequence before proceeding space characters detected!!!!')

  assert len(alternative_sequence) == len(sequence_structure), "Alternative sequence length doesn't match pdb sequence length, run interrupted!"

  sequence_structure = alternative_sequence
  print('... Alternative sequence loaded correctly')

print('... Target sequence:', sequence_structure)
#@markdown ****

In [ ]:
#@title <b><font color='#56b4e9'> MODEL RUN</font>
#@markdown Run this cell to evaluate the ΔG for the selected structure and sequence

#@markdown **N.B:** the ΔG value will be output in the scale of the chosen metric and also in kcal/mol (see the manuscript for how we converted the scale)
a=0.10413378327743603 ## fitting param from the manuscript to convert IF score scale to kcal/mol
b=0.6162549378400894 ## fitting param from the manuscript to convert IF score scale to kcal/mol

prob_tokens = run_model(coords_structure,sequence_structure,model,chain_target=chain_id)
aa_list, wt_scores = score_variants(sequence_structure,prob_tokens,alphabet)

dg_IF= np.nansum(wt_scores)
print('ΔG predicted (likelihoods sum): ',dg_IF)

dg_kcalmol= a * dg_IF + b

print('ΔG predicted (kcal/mol): ', dg_kcalmol)

aa_list_export=aa_list+['dG_IF','dG_kcalmol']
wt_scores_export=wt_scores+[dg_IF,dg_kcalmol]

df_export=pd.DataFrame({'Residue':aa_list_export,'score':wt_scores_export})

df_export.to_csv(f"outputs/"+f"{jobname}_dG_pos_scores_and_total.csv",sep=',')
## move pdb to output folder
try:
  os.rename(f"inputs/query_protein.pdb",f"outputs/{output_name_pdb}")
except:
  print('!!!! Data not saved, please re-upload the structure by running the uploading cell')


In [ ]:
#@title <b><font color='#56b4e9'>DOWNLOAD RESULTS </font></b>
#@markdown **N.B:** This will download **ALL** the predictions produced during the current session as zip file
os.system( "zip -r {} {}".format( f"dG_runs.zip" , f"outputs" ) )
files.download(f"dG_runs.zip")


<b><font color='#56b4e9'>EXTRA </font></b>

\\

**Output of the colab**

When a prediction is complete, the output files generated with the run are stored in the `/content/outputs` folder. When the <b><font color='#56b4e9'>DOWNLOAD RESULTS </font></b> cell is executed, all files are downloaded at once.

The output files stored for each run are
- the pdb used as input
- the fasta file of the query sequence
- the prediction output file

**Prediction file Format**

The output csv file consists of two columns, where the wild-type amino acid probability for each position is reported, and then at the bottom both the $\Delta G$ as the sum of the wild-type probabilities and in kcal/mol are reported.
\\

>OUTPUT FILE EXAMPLE:

>For a target protein with 45 residues, the scores file should be formatted like this:

>Residue  Likelihood

>M1               0.4  
A2                0.2
D3                0.3  
C5                0.9   
..  
..  
Y45               0.3
dG_IF       201
dG_kcalmol  13

\\

**Known problems:**

- Predictions on multi-domain proteins or proteins with complex folding kinetics show an absolute stability overestimated compared to the real one.

- Predictions are limited to proteins with 1023 residues (Max protein size for the ESM-IF language model)

\\

**License:**

The $ΔG$ predictor, and ESM-IF source code and parameters are licensed under the permissive Apache Licence, Version 2.0.

\\

**Bugs:**

For any bugs please report the issue on the project [Github](https://github.com/KULL-Centre/_2024_cagiada_stability) or contact one of the listed authors in the connected [manuscript](https://doi.org/).

\\

**Citing this work:**

If you use our model please cite:

citation here.

```

citation here

```
